# AI-powered Brochure Builder

- 🌍 Task: Generate a company brochure using its name and website for clients, investors, and recruits.
- 🧠 Model: Toggle `USE_OPENAI` to switch between OpenAI and Ollama models
- 🕵️‍♂️ Data Extraction: Scraping website content and filtering key links (About, Products, Careers, Contact).
- 📌 Output Format: a Markdown-formatted brochure streamed in real-time.
- 🚀 Tools: BeautifulSoup, OpenAI API, and IPython display, ollama.
- 🧑‍💻 Skill Level: Intermediate.

🛠️ Requirements
- ⚙️ Hardware: ✅ CPU is sufficient — no GPU required
- 🔑 OpenAI API Key 
- Install Ollama and pull llama3.2 or another lightweight model

In [3]:
import os
import requests
import json
import ollama
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import display, Markdown, update_display
from openai import OpenAI

In [4]:
# Load API key
load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')
if not api_key or not api_key.startswith('sk-'):
    raise ValueError("Invalid OpenAI API key. Check your .env file.")

# Define the model dynamically
USE_OPENAI = True  # True to use openai and False to use Ollama
MODEL = 'gpt-4o-mini' if USE_OPENAI else 'llama3.2:3b'

openai_client = OpenAI() if USE_OPENAI else None

In [5]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to scrape and process website content.
    """
    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        self.text = self.extract_text(soup)
        self.links = self.extract_links(soup)

    def extract_text(self, soup):
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            return soup.body.get_text(separator="\n", strip=True)
        return ""

    def extract_links(self, soup):
        links = [link.get('href') for link in soup.find_all('a')]
        return [link for link in links if link and 'http' in link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [6]:
class LLMClient:
    def __init__(self, model=MODEL):
        self.model = model

    def get_relevant_links(self, website):
        link_system_prompt = """
        You are given a list of links from a company website.
        Select only relevant links for a brochure (About, Company, Careers, Products, Contact).
        Exclude login, terms, privacy, and emails.

        ### **Instructions**
        - Return **only valid JSON**.
        - **Do not** include explanations, comments, or Markdown.
        - Example output:
        {
            "links": [
                {"type": "about", "url": "https://company.com/about"},
                {"type": "contact", "url": "https://company.com/contact"},
                {"type": "product", "url": "https://company.com/products"}
            ]
        }
        """

        user_prompt = f"""
        Here is the list of links on the website of {website.url}:
        Please identify the relevant web links for a company brochure. Respond in JSON format.
        Do not include login, terms of service, privacy, or email links.
        Links (some might be relative links):
        {', '.join(website.links)}
        """

        if USE_OPENAI:
            response = openai_client.chat.completions.create(
                model=self.model,
                messages=[
                    {"role": "system", "content": link_system_prompt},
                    {"role": "user", "content": user_prompt}
                ]
            )
            return json.loads(response.choices[0].message.content.strip())
        else:
            response = ollama.chat(
                model=self.model,
                messages=[
                    {"role": "system", "content": link_system_prompt},
                    {"role": "user", "content": user_prompt}
                ]
            )
            result = response.get("message", {}).get("content", "").strip()
            try:
                return json.loads(result)  # Attempt to parse JSON
            except json.JSONDecodeError:
                print("Error: Response is not valid JSON")
                return {"links": []}  # Return empty list if parsing fails


    def generate_brochure(self, company_name, content, language):
        system_prompt = """
        You are a professional translator and writer who creates fun and engaging brochures.
        Your task is to read content from a company’s website and write a short, humorous, joky,
        and entertaining brochure for potential customers, investors, and job seekers.
        Include details about the company’s culture, customers, and career opportunities if available.
        Respond in Markdown format.
        """

        user_prompt = f"""
        Create a fun brochure for '{company_name}' using the following content:
        {content[:5000]}
        Respond in {language} only, and format your response correctly in Markdown.
        Do NOT escape characters or return extra backslashes.
        """

        if USE_OPENAI:
            response_stream = openai_client.chat.completions.create(
                model=self.model,
                messages=[
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": user_prompt}
                ],
                stream=True
            )
            response = ""
            display_handle = display(Markdown(""), display_id=True)
            for chunk in response_stream:
                response += chunk.choices[0].delta.content or ''
                response = response.replace("```","").replace("markdown", "")
                update_display(Markdown(response), display_id=display_handle.display_id)
        else:
            response_stream = ollama.chat(
                model=self.model,
                messages=[
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": user_prompt}
                ],
                stream=True
            )
            display_handle = display(Markdown(""), display_id=True)
            full_text = ""
            for chunk in response_stream:
                if "message" in chunk:
                        content = chunk["message"]["content"] or ""
                        full_text += content
                        update_display(Markdown(full_text), display_id=display_handle.display_id)

In [7]:
class BrochureGenerator:
    """
    Main class to generate a company brochure.
    """
    def __init__(self, company_name, url, language='English'):
        self.company_name = company_name
        self.url = url
        self.language = language
        self.website = Website(url)
        self.llm_client = LLMClient()

    def generate(self):
        links = self.llm_client.get_relevant_links(self.website)
        content = self.website.get_contents()

        for link in links['links']:
            linked_website = Website(link['url'])
            content += f"\n\n{link['type']}:\n"
            content += linked_website.get_contents()

        self.llm_client.generate_brochure(self.company_name, content, self.language)

In [9]:
def main():
    company_name = "Tour Eiffel"
    url = "https://www.toureiffel.paris/fr"
    language = "English"

    generator = BrochureGenerator(company_name, url, language)
    generator.generate()

if __name__ == "__main__":
    main()


# 🗼 Join the Eiffel Frenzy! 🗼

## Welcome to the Tour Eiffel!
Get ready to reach new heights with the icon that stands tall (literally!) over Paris! Useful for both your Instagram and your self-esteem, the Eiffel Tower is open this summer from **9 AM to 11:45 PM**! 

### 🥖 Dining with a View!
Feeling hungry while contemplating the meaning of life at the 2nd floor? Dive into culinary perfection at **Le Jules Verne** or bask in the charming atmosphere of **Madame Brasserie**. Dinner with city views? Yeah, we thought so!

### 🎟️ Ticket-Tastic!
Heads up, savvy explorers: ticket demand is hotter than molten cheese on a freshly baked baguette! Secure your passes through our *official ticket booth*, or hop in line like it’s a roller coaster!

Here’s a pro tip: *Beware of the ticket trolls lurking about!* (seriously, they’re out there). Snag your tickets only through our official site to avoid mixing up your Eiffel adventure with a trip to your friend's basement.

### 🌱 Summer Vibes Galore!
Can’t make it inside? Don’t worry! Chill out in our **newly green terrace on the 1st floor**—perfect for Instagram pics and existential musings. The **Eiffel Effect** here is as real as your desire for selfies.

### 🌟 Perfect Gift Alert!
Looking for that unique gift that says *I love you, but I also love Parisian architecture*? Check out our **E-Shop** for Eiffel merchandise that will make your friends think you’re an elite traveler.

### 🌍 A Tower of Opportunities!
Are you a company looking to host a jaw-dropping event? Our **Gustave Eiffel Lounge** is ready to impress a crowd or simply invite your favorite work buddies (we won't tell!).

### 🔗 Keep in Touch!
For all the latest gossip (and chocolate croissant recipes), follow us on social media: Facebook, Instagram, and TikTok! We promise no cat videos, only stunning shots of our beloved monument!

### 🏗️ Wanna Join the Tower Family?
Enthusiastic about joining our adventure? We’re always on the lookout for passionate souls! Check our **Careers Page**—where your dreams of working under the Parisian sky could come true!

---

**Don’t miss out on your chance to embrace the romantic vibes of Paris from atop the world's most famous tower!** After all, who says you can’t have fun and enjoy a stunning view at the same time?

🗼 See you at the top! 🎉
